# PROYECTO PYTHON CORTE 2 - 2019-2
## Integrantes
### - Cristian Duran
### - Kelly Fernanda Vasquez
### - Luis Carlos Jordan

## DECLARACIÓN DE VARIABLES Y DATOS PÚBLICOS

In [1]:
# Librerias que utiliza este aplicativo FLASK.
from flask import Flask, render_template, request, make_response, escape, url_for, redirect, flash, Response
from flask_debug import Debug
# Librerias que utiliza el Web scraper
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np
import csv
import mysql.connector as mariadb
import hashlib
# Libreria para convertir a imagen
import io
app = Flask(__name__)
app.secret_key = '123456'

csv_path = './static/futbol.csv'
csv_obj = csv.DictReader(open(csv_path, 'r'))
csv_list = list(csv_obj)

url = 'https://resultados.as.com/resultados/futbol/primera_femenina/clasificacion/'

def Conexion():
    return mariadb.connect(host='localhost', port='3306', user='root', password='123456', database='proyecto_python2')

## DECLARACIÓN DE RUTAS

### INDEX - PÁGINA PRINCIPAL

In [2]:
@app.route("/")
def index():
    return render_template("index.php")

### FUNCIONES WEBSCRAPING

In [3]:
@app.route("/webscraper")
def webscraper():
    equipos = []
    equipos = WebScraper()
    partidos = []
    partidos = WebScraper()
    return render_template('datos_webscraper.php', nombre_equipos=equipos, partidos_jugados=partidos )

def WebScraper():
    page =requests.get(url)
    soup = (BeautifulSoup(page.content, 'html.parser'))
    
    equipos = []
    
    equipos_futbol = soup.find_all('span', class_='nombre-equipo') 
    #equipos = list()
    count = 0
    for i in equipos_futbol:
        if count < 6:
            equipos.append(i.text)
        else:
            break
        count += 1
    print(equipos, len(equipos))
    
    partidos = []
    
     #Puntos por partidos jugados
    partido = soup.find_all('td', class_='destacado') 
    #partidos = list()

    count = 0
    for i in partido:
        if count < 6:
           partidos.append(i.text)
        else:
            break
        count += 1

    #print(partidos, len(partidos))
    
    return equipos, partidos

@app.route("/graficas_scraping")
def graficas_scraping():
    return  render_template('graficas_scraping.php')  

### FUNCIONES CVS

In [ ]:
@app.route("/csv")
def csv():
    datos = pd.read_csv('./static/futbol.csv')
    df = pd.DataFrame(datos)
    
    #Grafica de Lineas
    grafico_cantidad = df['Puntos de Clasificacion']
    grafico_relacion = df[['Nombre Equipo','Puntos de Clasificacion']]
    plt.suptitle('Lista de Equipos de Futbol Liga Femenina Española')
    plt.ylabel('Puntos')
    figura= plt.plot(grafico_cantidad)
    plt.savefig("./static/images/graficas/lineas_csv.png", dpi=100, bbox_inches='tight')
    #figura = plt.gcf()
    plt.show()
    plt.draw()
    
    #Grafica de Torta
    datosnew = df.iloc[1:6]
    grafico_cantidad1 = datosnew['Puntos de Clasificacion']
    grafico_relacion1 = datosnew[['Nombre Equipo','Puntos de Clasificacion']]
    nombre_equipo = grafico_relacion1["Nombre Equipo"]
    puntos_clasificacion = grafico_relacion1["Puntos de Clasificacion"]
    explode = None 
    plt.title("Pocisiones Primero 6 Equipos -  Liga Femenina Española")
    plt.pie(puntos_clasificacion, explode=None, labels=nombre_equipo, 
    autopct='%1.1f%%', shadow=True, startangle=140)
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig("./static/images/graficas/pie_csv.png", dpi=100, bbox_inches='tight')
    #figura = plt.gcf()
    plt.show()
    plt.draw()
    
    #Grafico de Barras
    fig = plt.figure('Gráfica de barras') # Figure
    ax = fig.add_subplot(111) # Axes
    df.groupby('Nombre Equipo')['Puntos de Clasificacion'].sum().plot(kind='barh', legend='Reverse')
    plt.xlabel('Puntos Clasificacion ')
    explode = None
    plt.figure(figsize=(100,60))
    plt.savefig("./static/images/graficas/barras_csv.png", dpi=100, bbox_inches='tight')
    plt.draw()
    plt.show()
    
    return  render_template('datos_csv.php', object_list=csv_list,)   

@app.route("/graficas_csv")
def graficas_csv():
    return  render_template('graficas_csv.php')  


### FUNCIONES BASES DE DATOS

In [ ]:
@app.route("/basedatos")
def basedatos():
    datos_db = []

    conexion = Conexion()
    cursor = conexion.cursor()
    try:
        cursor.execute("SELECT nombreequipos, puntuacion FROM proyecto2")
        for nombreequipos, puntuacion in cursor:
            datos_db = [nombreequipos, puntuacion]
    except mariadb.Error as error:
        print("Error: {}".format(error))
    
    #Guardar Registros en CSV
    #with open('DATOS_PRUEBA_SQL.csv', 'w', newline='') as f_handle:
     #   writer = csv.writer(f_handle)
      #  for row in filas:
       #     writer.writerow(row)
    
    conexion.close()
       
    return render_template('datos_sql.php', db_datos=datos_db)

@app.route("/graficas_bd")
def graficas_bd():
    return  render_template('graficas_bd.php') 



### FINAL

In [ ]:
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Oct/2019 23:46:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2019 23:46:45] "GET /graficas_webscraping HTTP/1.1" 404 -
127.0.0.1 - - [21/Oct/2019 23:46:45] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Oct/2019 23:46:49] "GET /graficas_csv HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2019 23:46:49] "GET /static/images/graficas/pie_csv.png HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2019 23:46:49] "GET /static/images/graficas/lineas_csv.png HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2019 23:46:49] "GET /static/images/graficas/barras_csv.png HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2019 23:46:55] "GET /basedatos HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2019 23:46:57] "GET /graficas_bd HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2019 23:46:57] "GET /static/images/graficas/barras_csv.jpg HTTP/1.1" 404 -
127.0.0.1 - - [21/Oct/2019 23:47:00] "GET /graficas_csv HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2019 23:47:08] "GET /graficas_webscrap

['FC Barcelona Femenino', 'Deportivo de La Coruña Femenino', 'Atlético de Madrid Femenino', 'Levante UD Femenino', 'EDF Logroño', 'Rayo Vallecano Femenino'] 6


127.0.0.1 - - [21/Oct/2019 23:47:13] "GET /webscraper HTTP/1.1" 200 -


['FC Barcelona Femenino', 'Deportivo de La Coruña Femenino', 'Atlético de Madrid Femenino', 'Levante UD Femenino', 'EDF Logroño', 'Rayo Vallecano Femenino'] 6
